In [2]:
import os, shutil

In [19]:
original_dataset_dir = '/home/dsc/tfm_data/dogs-vs-cats/train'

In [6]:
!pwd

/home/dsc/TFM_food_recognition


Creamos el directorio donde vamos a guardar un sample de los datos:

In [7]:
base_dir = '/home/dsc/tfm_data/cats_and_dogs_small'
os.mkdir(base_dir)


Creamos los directorios de training, validation y test:

In [8]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

Directorio de training de las fotos de gatitos:

In [9]:
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

Directorio de training de las fotos de perros:

In [10]:
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

Directorio de validación de las fotos de gatitos:

In [11]:
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

Directorio de validación de las fotos de los perros:

In [12]:
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

Y ahora vamos con el directorio de test de los gatos y los perros:

In [13]:
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

In [14]:
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

Vamos a copiar las mil primeras imágenes al directorio de train de gatitos, las siguietnes 500 al de validation y las restantes 500, al de test:

In [20]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

In [21]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

In [22]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

Realizamos la misma operación con los directorios de perros: